# Test data loading for static dataset

This is a short test to show how data loading works 

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

# all paths relative to afa directory 
sys.path.insert(0, os.path.abspath('../../afa'))

### Load normal dataset

In [2]:
from afa.data_modelling.datasets.data_loader.data_loader_static import DataLoader_static

In [3]:
data_dir = '../../../data/static/synthetic_1/'
data_file = data_dir + 'synthetic_1.csv' 
superfeature_mapping_file = data_dir + 'superfeatures.csv'
# superfeature_mapping_file = None
    
data_loader = DataLoader_static( data_file     = data_file,
                                 superfeature_mapping_file  = superfeature_mapping_file )

dataset = data_loader.load() 

## Preprocessing

In [4]:
from afa.data_modelling.preprocessing import Normalizer
from afa.data_modelling.utils.model_utils import PipelineComposer

2022-12-28 12:12:30.598077: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-28 12:12:30.719413: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-28 12:12:30.719430: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-28 12:12:31.496054: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [5]:
normalization = 'minmax'
normalizer = Normalizer(normalization)

# Data preprocessing
filter_pipeline = PipelineComposer(normalizer)

dataset = filter_pipeline.fit_transform(dataset)

print('Finish preprocessing.')

Finish preprocessing.


### Define problem

Set the prediction problem that we want to solve. Set the problem (one-shot), set the label and set the treatment features (not supported). 

We also define the metric for evaluation and the task itself (whether classification or regression
  - problem: 
    - 'classification'
  - label_name: the column name for the label(s)
  - treatment: the column name for treatments

In [6]:
from afa.data_modelling.problem.problem_static import ProblemMaker_static

In [7]:
label_name = 'Y'
problem = 'classification'
treatment = None

# Define problem 
problem_maker = ProblemMaker_static(problem=problem, label=[label_name], treatment=treatment)

dataset  = problem_maker.fit_transform(dataset)

## Impute dataset

In [8]:
from afa.data_modelling.imputation import Imputation

ModuleNotFoundError: No module named 'data_modelling.imputation'

In [ ]:
# Set imputation models
imputation_model = 'median'

# Impute the missing data
imputation          = Imputation(imputation_model_name = imputation_model)

imputation_pipeline = PipelineComposer(imputation)

dataset_train = imputation_pipeline.fit_transform(dataset_train)
dataset_test  = imputation_pipeline.transform(dataset_test)

print('Finish imputation.')

## Perform prediction

In [ ]:
# Train the predictive model
task = 'classification'
pred_class = prediction_miss(   classification_params       = configs['classifier']['class_params'], 
                                task          = task,
                                imp_params    = configs['classifier']['imp_params'],  
                                miss_params   = configs['classifier']['miss_params'])
pred_class.fit(dataset_train, 
               fold=0, 
               train_split=configs['data_splits']['models']['classifier_train'], 
               valid_split=configs['data_splits']['models']['classifier_test'])

# performance on given dataset:
Y_tilde_test     = pred_class.predict(dataset_train, fold=0, 
                                      test_split= configs['data_splits']['models']['classifier_test'])
_, Y_test , _    = dataset_train.get_fold(fold=0, 
                                          split = configs['data_splits']['models']['classifier_test']) 

accuracy = accuracy_score( Y_tilde_test.round() , Y_test.round())
print("Accuracy on testset: ", accuracy)